#NPC Chatbot App with ChatGPT, LangChain and Streamlit

#Install Dependencies

In [ ]:
# Install Dependencies
!pip install langchain==0.1.12
!pip install langchain-openai==0.0.8
!pip install langchain-community==0.0.29
!pip install streamlit==1.32.2
!pip install chromadb==0.4.24
!pip install pyngrok==7.1.5
!pip install nltk

#Load OpenAI API Credentials

In [ ]:
# Load OpenAI API Credentials
from getpass import getpass


OPENAI_KEY = getpass('Enter Open AI API Key: ')


#Set Environment Variable

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = OPENAI_KEY

#Write App Code Header

In [ ]:
# Install Dependencies (No need to re-run if already installed in your Colab)
# ... (dependency installations)

# Load OpenAI API Credentials (No need to re-run if already set)
# ... (API key loading)

# Write App Code - REPLACE YOUR EXISTING app.py CONTENT WITH THIS
%%writefile app.py
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
from langchain.callbacks.base import BaseCallbackHandler
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores.chroma import Chroma
from langchain.chains import LLMChain
from operator import itemgetter

import streamlit as st
import tempfile
import os
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt')

# Customize initial app landing page
st.set_page_config(page_title="NPC Simulation Chatbot", page_icon="🎭")
st.title("NPC Simulation Chatbot 🎭")
st.sidebar.header("NPC Character Document Upload")

@st.cache_resource(ttl="1h")
def configure_npc_agent(uploaded_file):
    if uploaded_file is not None:
        text_content = uploaded_file.read().decode("utf-8")

        # --- Normalize line endings to standard Unix-style (\n) ---
        text_content = text_content.replace('\\r\\n', '\\n').replace('\\r', '\\n')

        # --- NEW DEBUGGING: Print the raw text_content ---
        st.write("--- **RAW TEXT CONTENT FROM UPLOADED FILE (Line endings normalized)** ---")
        st.write("<pre>" + text_content + "</pre>", unsafe_allow_html=True) # Use <pre> for formatting
        st.write("--- **END RAW TEXT CONTENT** ---")
        # --- END NEW DEBUGGING ---


        # --- NEW SECTION PARSING: Line-based parsing ---
        lines = text_content.strip().split('\n') # Split into lines
        sections_dict = {}
        current_section_name = None
        current_section_content = []

        for line in lines:
            line = line.strip()
            if line.startswith("---") and line.endswith("---"):
                if current_section_name:
                    sections_dict[current_section_name] = "\n".join(current_section_content).strip()
                    current_section_content = [] # Reset content for new section
                current_section_name = line.strip('-').strip() # Extract section name
            elif current_section_name:
                current_section_content.append(line) # Append line to current section

        if current_section_name and current_section_content: # Capture last section
            sections_dict[current_section_name] = "\n".join(current_section_content).strip()


        # --- DEBUGGING: Print the sections_dict ---
        st.write("--- **DEBUGGING: SECTIONS DICTIONARY AFTER LINE-BASED PARSING** ---")
        st.write(sections_dict) # Print the sections dictionary
        st.write("--- **END DEBUGGING: SECTIONS DICTIONARY** ---")
        # --- END DEBUGGING ---


        character_description = sections_dict.get("Character Description", "")
        backstory_world_info = sections_dict.get("Backstory and World Info", "")
        quests_text = sections_dict.get("Quests", "")

        quests = [q.strip() for q in quests_text.strip().split("- ") if q.strip()]


        # --- DEBUGGING OUTPUT USING st.write() ---
        st.write("--- **DEBUGGING INFORMATION - SECTIONS AFTER PROCESSING** ---")
        st.write("**Character Description Section:**")
        st.write(character_description)
        st.write("**Backstory and World Info Section:**")
        st.write(backstory_world_info)
        st.write("**Quests Section:**")
        st.write(quests_text)
        st.write("**Extracted Quests List:**")
        st.write(quests)
        st.write("--- **END DEBUGGING INFORMATION** ---")
        # --- END DEBUGGING OUTPUT ---


        # Split backstory and world info into chunks for RAG
        text_splitter = CharacterTextSplitter(
            separator="\n",
            chunk_overlap=200,
            chunk_size=2000,
            length_function=len,
        )
        doc_chunks = text_splitter.create_documents([backstory_world_info])

        # Create embeddings and vectorstore for backstory/world info
        embeddings_model = OpenAIEmbeddings()
        vectordb = Chroma.from_documents(doc_chunks, embeddings_model)
        retriever = vectordb.as_retriever()

        return character_description, retriever, quests

    return None, None, None


# Manages live updates to a Streamlit app's display by appending new text tokens
class StreamHandler(BaseCallbackHandler):
    def __init__(self, container, initial_text=""):
        self.container = container
        self.text = initial_text

    def on_llm_new_token(self, token: str, **kwargs) -> None:
        self.text += token
        self.container.markdown(self.text)

# UI element to upload text document
uploaded_file = st.sidebar.file_uploader(
    label="Upload NPC Character Text Document", type=["txt"]
)

if uploaded_file is None:
    st.info("Please upload your NPC Character Text Document to continue.")
    st.stop()

# Configure NPC agent
character_description, retriever, quests = configure_npc_agent(uploaded_file)

if character_description is None or retriever is None or quests is None:
    st.error("Error loading character document. Please check the format.")
    st.stop()

# Load LLM
chatgpt = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.7, streaming=True)

# System prompt for NPC character
system_prompt_template = SystemMessagePromptTemplate.from_template(character_description)

# RAG Prompt for retrieving NPC/World info
rag_prompt_template = ChatPromptTemplate.from_messages([
    system_prompt_template,
    HumanMessagePromptTemplate.from_template("""Use the following context to answer the user question about yourself or the world.
                                              Context: {context}
                                              Question: {question}
                                              If you cannot answer from the context, just say you do not know.
                                              Answer in character as described in the system prompt.""")
])

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

rag_chain = (
    {
        "context": itemgetter("question") | retriever | format_docs,
        "question": itemgetter("question")
    }
    | rag_prompt_template
    | chatgpt
)


# General conversation prompt (no RAG)
conversation_prompt_template = ChatPromptTemplate.from_messages([
    system_prompt_template,
    HumanMessagePromptTemplate.from_template("{question}")
])

conversation_chain = conversation_prompt_template | chatgpt


# Quest related prompts
quest_suggestion_prompt_template = ChatPromptTemplate.from_messages([
    system_prompt_template,
    HumanMessagePromptTemplate.from_template("Suggest a quest for the user, keeping in character. If no quests are appropriate, just say 'No quests available right now.'")
])
quest_suggestion_chain = quest_suggestion_prompt_template | chatgpt


# Conversation Handling
streamlit_msg_history = StreamlitChatMessageHistory(key="npc_chat_messages")

if len(streamlit_msg_history.messages) == 0:
    streamlit_msg_history.add_ai_message("Greetings! How can I help you today?")

for msg in streamlit_msg_history.messages:
    st.chat_message(msg.type).write(msg.content)

if user_prompt := st.chat_input():
    st.chat_message("human").write(user_prompt)

    # Determine intent: quest, info, or general conversation
    if "quest" in user_prompt.lower():
        if "give me a quest" in user_prompt.lower() or "offer quest" in user_prompt.lower() or "i want a quest" in user_prompt.lower():
            if quests:
                quest_to_offer = quests.pop(0) # Get the first quest and remove it from the list
                with st.chat_message("ai"):
                    stream_handler = StreamHandler(st.empty())
                    config = {"callbacks": [stream_handler]}
                    response = conversation_chain.invoke({"question": f"Offer the user the following quest: {quest_to_offer}"}, config) # Use conversation chain to offer quest in character
                streamlit_msg_history.add_user_message(user_prompt)
                streamlit_msg_history.add_ai_message(response.content)

            else:
                 with st.chat_message("ai"):
                    stream_handler = StreamHandler(st.empty())
                    config = {"callbacks": [stream_handler]}
                    response = quest_suggestion_chain.invoke({}, config)
                 streamlit_msg_history.add_user_message(user_prompt)
                 streamlit_msg_history.add_ai_message(response.content)


        elif "what quests" in user_prompt.lower() or "tell me about quests" in user_prompt.lower():
            if quests:
                quest_list_str = "\n".join([f"- {q}" for q in quests])
                response_text = f"Here are the quests I know:\n{quest_list_str}"
            else:
                response_text = "I don't have any quests for you right now."

            with st.chat_message("ai"):
                st.write(response_text)
            streamlit_msg_history.add_user_message(user_prompt)
            streamlit_msg_history.add_ai_message(response_text)


        else: # Assume general quest related query
            with st.chat_message("ai"):
                stream_handler = StreamHandler(st.empty())
                config = {"callbacks": [stream_handler]}
                response = conversation_chain.invoke({"question": user_prompt}, config) # General conversation in character
            streamlit_msg_history.add_user_message(user_prompt)
            streamlit_msg_history.add_ai_message(response.content)


    elif "tell me about" in user_prompt.lower() or "who are you" in user_prompt.lower() or "where am i" in user_prompt.lower() or "what is this place" in user_prompt.lower():
        # RAG for NPC/World info
        with st.chat_message("ai"):
            stream_handler = StreamHandler(st.empty())
            config = {"callbacks": [stream_handler]}
            response = rag_chain.invoke({"question": user_prompt}, config)
        streamlit_msg_history.add_user_message(user_prompt)
        streamlit_msg_history.add_ai_message(response.content)

    else:
        # General conversation
        with st.chat_message("ai"):
            stream_handler = StreamHandler(st.empty())
            config = {"callbacks": [stream_handler]}
            response = conversation_chain.invoke({"question": user_prompt}, config)
        streamlit_msg_history.add_user_message(user_prompt)
        streamlit_msg_history.add_ai_message(response.content)




#Starting the Streamlit App

In [ ]:
# Starting the Streamlit App
!streamlit run app.py --server.port=8989 &>/./logs.txt &

#Setting Up ngrok Tunnel

In [ ]:
# Setting Up ngrok Tunnel
from getpass import getpass

ngrok_auth_token = getpass('Enter ngrok API Key: ')

In [ ]:
from pyngrok import ngrok
import yaml

# Terminate open tunnels if exist
ngrok.kill()

# Authenticate ngrok with the token read from the file
!ngrok config add-authtoken {ngrok_auth_token}

# Open an HTTPS tunnel on port XXXX which you get from your `logs.txt` file
ngrok_tunnel = ngrok.connect(8989)
print("Streamlit App:", ngrok_tunnel.public_url)